<a href="https://colab.research.google.com/github/tatirBarbo/ImersaoIA_DiagnosticoTDAH_TEA/blob/main/Projeto_Diagnostico_Preliminar_TDHA_TEA_Para_Pais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72

In [5]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [6]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [7]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [8]:
##################################################
# --- Agente 1: Coletor de Dados atualizados --- #
##################################################
def agente_coletor(transtorno, idade, data_de_hoje):

  coletor = Agent(
      name = "agente_coletor",
      model = "gemini-2.0-flash",
      description = "Agente que busca informações refetente aos comportamentos de crianças com TDAH ou TEA no Google",
      tools = [google_search],
      instruction = """
      Você é um assitente de pesquisa, especialista em TDAH (Transtorno de Déficit de Atenção e Hiperatividade) e TEA (Transtorno do Espectro Autista). A sua tarefa é usar a ferramenta de busca do google (google_search)
      para recuperar as informações mais recentes e relevantes sobre crianças diagnosticadas com o tipo de transtorno abaixo de acordo com a idade informada.
      Foque no máximo nas 10 informações mais relevantes, com base nas características mais comuns de estarem presentes nas crianças que são diagnosticas com o transtorno informado e que são mais faceis de serem percebidas
      pelos pais e pessoas que convivem com as crianças diagnosticas, de acordo com a idade informada.
      Se uma característica for mais rara de estar presente, mas ainda sim ser muito relevante para o diagnóstico, pois é uma caracterísca muito citada pelos especialistas, retornar a característica identificando como
      uma característica forte para apoiar no o diagnóstico.
      Essas informações relevantes, comuns de estarem presentes e fáceis de serem identificadas, devem ser atuais, de no máximo um ano antes da data de hoje.
      """
  )

  entrada_do_agente_coletor = f"Transtorno (1 = TDAH, 2 = TEA): {transtorno}\n Idade: {idade}\nData de hoje: {data_de_hoje}"

  informacoes_coletadas = call_agent(coletor, entrada_do_agente_coletor)
  return informacoes_coletadas

In [9]:
################################################
# --- Agente 2: Agente Questionador  --- #
################################################
def agente_questionador(transtorno, idade, informacoes_coletadas):
    questionador = Agent(
        name="agente_questionador",
        model="gemini-2.0-flash",
        # Instruções do Agente Questionador #################################################
        instruction="""
        Você é um gerador de perguntas, especialista em criar perguntas classificando elas por relevância. Com base na lista de informações mais recentes e relevantes coletadas, você deve:
        usar a ferramenta de busca do Google (google_search) para formular um questionário com no máximo 30 perguntas sobre características e comportamentos para ajudar a diagnosticar se uma criança tem grandes ou pequenas
        chances, de ter o transtorno informado, de acordo com a idade dela.
        Você deve classificar a relevância das perguntas para o diagnóstico, classificando de 1 a 5, onde 1 é a menos relevante e a 5 é a mais relevante, para auxiliar posteriomente no diagnostico.
        Você também pode usar o (google_search) para encontrar mais informações sobre as características e relevância para realizar a classificação.
        Como retorno de todo o levantamento, você deverá escolher e retornar apenas as 10 perguntas, com mais relevância entre as perguntas geradas inicialmente, para serem respondidas.
        Você deve criar as 10 perguntas utilizando uma linguagem simples e direta, tendo como princípio que a pessoa que irá responder é leiga no assunto.
        Esse agente não deve retornar mais perguntas ou as informações que serviram de insumo para gerar as 10 questões.
        """,
        description="Agente que gera e classifica perguntas",
        tools=[google_search]
    )

    entrada_do_agente_questionador = f"Transtorno (1 = TDAH, 2 = TEA):{transtorno}\n Idade: {idade}\n Informacoes coletadas: {informacoes_coletadas}"
    # Executa o agente
    questionario = call_agent(questionador, entrada_do_agente_questionador)
    return questionario

In [10]:
######################################
# --- Agente 3: Avaliador --- #
######################################
def agente_avaliador(transtorno, idade, questionario, respostas_do_questionario):

    avaliador = Agent(
        name="agente_avaliador",
        model="gemini-2.0-flash",
        instruction="""
            Você é um avaliador especialista em TDAH (Transtorno de Déficit de Atenção e Hiperatividade) e TEA (Transtorno do Espectro Autista), que utiliza linguagem simples e fácil para pessoas leigas nos assuntos TDAH e TEA.
            Utilize as informações coletadas, as respostas fornecidas para o questionário e análise a resposta dada para cada pergunta, levando em consideração a relevância da pergunta para o diagnóstico.
            As opções de respostas para cada pergunta são 1 - Apresenta o comportamento/característica, 2 - NÃO apresenta o comportamento/característica, 3 - Apresenta as vezes ou 4 - Não consigo responder.
            Com base nas resposta e relevância da pergunta, avalie a probabilidade da criança ter algum grau do transtorno analisado. Forneça um resumo da análise feita e a conclusão da análise.
            Utilize um texto curto, resumido e simples, porém explicativo, como se estivesse explicando para uma pessoa leiga no assunto.
            Forneça também a probabilidade de acerto do diagnóstico.
            Forneça no final os próximos passos e fontes de referências especializadas no assunto.
            """,
        description="Agente avaliador das informações fornecidas para retornar um possível diagnóstico."
    )
    entrada_do_agente_avaliador = f"Transtorno (1 = TDAH, 2 = TEA):{transtorno}\n Idade: {idade}\n Perguntas: {questionario}\n Respostas: {respostas_do_questionario}"
    # Executa o agente
    diagnostico = call_agent(avaliador, entrada_do_agente_avaliador)
    return diagnostico


In [16]:
hoje = date.today().strftime("%d/%m/%Y")

print("Iniciando o Sistema de Apoio Inicial ao Diagnóstico de TDAH e TEA para crianças de 3 a 10 anos. \n")

# --- Obter o Tipo de Transtorno  ---
tipo_transtorno = input("Por favor, informe o tipo de transtorno, sendo 1 para TDAH, 2 para TEA e 3 para Sair: ")
while not tipo_transtorno:
  print("Você esqueceu de digitar o tipo de transtorno!")
  tipo_transtorno = input("Por favor, informe o tipo de transtorno, sendo 1 para TDAH, 2 para TEA e 3 para Sair: ")

if tipo_transtorno == "1":
   tipo = "Transtorno de Déficit de Atenção e Hiperatividade - TDAH"
elif tipo_transtorno == "2":
   tipo = "Transtorno do Espectro Autista - TEA"
elif tipo_transtorno == "3":
   tipo = "Sair"
else:
  while tipo_transtorno not in ["1", "2", "3"] or not tipo_transtorno:
     print("O valor informado não é válido!")
     print("\n")
     tipo_transtorno = input("Informe um tipo de transtorno válido, sendo 1 = TDAH, 2 = TEA e 3 = Sair: ")
     tipo = tipo_transtorno

if tipo =="Sair":
  print("\n Obrigada por utilizar o sistema de Apoio Inicial ao Diagnóstico de TDAH e TEA para crianças de 3 a 10 anos.")
  print("\n Fim do programa!")
else:
  # --- Obter o Idade da criança  ---
  idade_crianca = input("Por favor, informe a idade da criança: ")

  while not idade_crianca or idade_crianca not in ["3", "4", "5", "6", "7", "8", "9", "10"]:
    print("A idade informada não é válida! A idade deve estar entre 3 e 10 anos.")
    print("\n")
    idade_crianca = input("Por favor, informe a idade da criança: ")

  print("\n--- coletando informações ---\n")
  informacoes_coletadas = agente_coletor(tipo, idade_crianca, hoje)

  questionario = agente_questionador(tipo, idade_crianca, informacoes_coletadas)
  print("\n--- Questionário ---\n")
  display(to_markdown(questionario))

  print("\n Responda as perguntas acima na ordem. Valores: \n 1 - Apresenta o comportamento/característica \n 2 - NÃO apresenta o comportamento/característica \n 3 - Apresenta as vezes \n 4 - Não consigo responder. ")
  resposta_1 = input("Pergunta 1: ")
  while not resposta_1 or resposta_1 not in ["1", "2", "3", "4"]:
     resposta_1 = input("Valor inválido. Responda novamente. \n Pergunta 1: ")

  resposta_2 = input("Pergunta 2: ")
  while not resposta_2 or resposta_2 not in ["1", "2", "3", "4"]:
     resposta_2 = input("Valor inválido. Responda novamente. \n Pergunta 2: ")

  resposta_3 = input("Pergunta 3: ")
  while not resposta_3 or resposta_3 not in ["1", "2", "3", "4"]:
     resposta_3 = input("Valor inválido. Responda novamente. \n Pergunta 3: ")

  resposta_4 = input("Pergunta 4: ")
  while not resposta_4 or resposta_4 not in ["1", "2", "3", "4"]:
     resposta_4 = input("Valor inválido. Responda novamente. \n Pergunta 4: ")

  resposta_5 = input("Pergunta 5: ")
  while not resposta_5 or resposta_5 not in ["1", "2", "3", "4"]:
     resposta_5 = input("Valor inválido. Responda novamente. \n Pergunta 5: ")

  resposta_6 = input("Pergunta 6: ")
  while not resposta_6 or resposta_6 not in ["1", "2", "3", "4"]:
     resposta_6 = input("Valor inválido. Responda novamente. \n Pergunta 6: ")

  resposta_7 = input("Pergunta 7: ")
  while not resposta_7 or resposta_7 not in ["1", "2", "3", "4"]:
     resposta_7 = input("Valor inválido. Responda novamente. \n Pergunta 7: ")

  resposta_8 = input("Pergunta 8: ")
  while not resposta_8 or resposta_8 not in ["1", "2", "3", "4"]:
     resposta_8 = input("Valor inválido. Responda novamente. \n Pergunta 8: ")

  resposta_9 = input("Pergunta 9: ")
  while not resposta_9 or resposta_9 not in ["1", "2", "3", "4"]:
     resposta_9 = input("Valor inválido. Responda novamente. \n Pergunta 9: ")

  resposta_10 = input("Pergunta 10: ")
  while not resposta_10 or resposta_10 not in ["1", "2", "3", "4"]:
     resposta_10 = input("Valor inválido. Responda novamente. \n Pergunta 10: ")

  print("\n--- Análise das informações ---\n")
  avaliacao = agente_avaliador(tipo, idade_crianca, questionario, [resposta_1, resposta_2, resposta_3, resposta_4, resposta_5, resposta_6, resposta_7, resposta_8, resposta_9, resposta_10])
  display(to_markdown(avaliacao))
  print("-------------------------------------------------------------------")

  print("\n Obrigada por utilizar o sistema de Apoio Inicial ao Diagnóstico de TDAH e TEA para crianças de 3 a 10 anos.")
  print("\n Fim do programa!")

Iniciando o Sistema de Apoio Inicial ao Diagnóstico de TDAH e TEA para crianças de 3 a 10 anos. 

Por favor, informe o tipo de transtorno, sendo 1 para TDAH, 2 para TEA e 3 para Sair: 1
Por favor, informe a idade da criança: 11
A idade informada não é válida! A idade deve estar entre 3 e 10 anos.


Por favor, informe a idade da criança: 2
A idade informada não é válida! A idade deve estar entre 3 e 10 anos.


Por favor, informe a idade da criança: 
A idade informada não é válida! A idade deve estar entre 3 e 10 anos.


Por favor, informe a idade da criança: 3

--- coletando informações ---


--- Questionário ---



> Com base nas informações fornecidas sobre TDAH em crianças de 3 anos, formulei as seguintes perguntas para auxiliar na identificação de possíveis sinais do transtorno, classificadas por relevância (1 a 5, sendo 5 a mais relevante):
> 
> 
> Aqui estão as 10 perguntas mais relevantes, formuladas de maneira clara e direta:
> 
> 1.  Seu filho(a) parece estar sempre "ligado(a", como se tivesse um motorzinho, mesmo quando precisa ficar quieto(a)? (Relevância: 5)
> 2.  Seu filho(a) tem dificuldade em esperar a vez dele(a) em brincadeiras ou atividades? (Relevância: 5)
> 3.  Seu filho(a) frequentemente interrompe conversas ou atividades de outras pessoas? (Relevância: 4)
> 4.  Seu filho(a) tem dificuldade em prestar atenção em atividades por muito tempo, mesmo em brincadeiras que ele(a) goste? (Relevância: 4)
> 5.  Seu filho(a) se distrai facilmente com barulhos ou coisas acontecendo ao redor dele(a)? (Relevância: 4)
> 6.  Seu filho(a) tem dificuldade em seguir instruções ou terminar tarefas que você pede? (Relevância: 4)
> 7.  Seu filho(a) fala muito, mesmo quando não é apropriado? (Relevância: 3)
> 8.  Seu filho(a) parece esquecer coisas que você acabou de falar ou pedir para ele(a) fazer? (Relevância: 3)
> 9.  Seu filho(a) tem dificuldade em brincar quieto(a) ou fazer atividades calmas? (Relevância: 3)
> 10. Você observa esses comportamentos do seu filho(a) em casa e também em outros lugares, como na creche ou em parques? (Relevância: 5)
> 



 Responda as perguntas acima na ordem. Valores: 
 1 - Apresenta o comportamento/característica 
 2 - NÃO apresenta o comportamento/característica 
 3 - Apresenta as vezes 
 4 - Não consigo responder. 
Pergunta 1: 1
Pergunta 2: 3
Pergunta 3: 1
Pergunta 4: 1
Pergunta 5: 3
Pergunta 6: 3
Pergunta 7: 3
Pergunta 8: 3
Pergunta 9: 3
Pergunta 10: 1

--- Análise das informações ---



> ## Análise Preliminar de Possível TDAH em Criança de 3 Anos
> 
> Olá! Vamos analisar juntos as respostas do questionário para entender melhor se seu filho(a) apresenta sinais sugestivos de Transtorno de Déficit de Atenção e Hiperatividade (TDAH).
> 
> **Resumo da Análise:**
> 
> As respostas indicam que seu filho(a) **apresenta alguns comportamentos** que podem estar associados ao TDAH. Observamos que em questões de alta relevância, como a criança estar sempre "ligada" e a ocorrência dos comportamentos em diferentes ambientes, a resposta foi positiva. Em outras questões, as respostas indicam que o comportamento é apresentado as vezes.
> 
> É importante lembrar que em crianças muito novas, como aos 3 anos, alguns comportamentos como dificuldade em esperar a vez ou se distrair podem ser comuns do desenvolvimento. No entanto, a frequência e intensidade com que esses comportamentos ocorrem, e o quanto eles atrapalham o dia a dia da criança, é que nos ajudam a identificar se há algo a mais acontecendo.
> 
> **Conclusão:**
> 
> Com base nas respostas fornecidas, existe uma **possibilidade** de que seu filho(a) apresente algum grau de TDAH. No entanto, **é fundamental ressaltar que este questionário não é um diagnóstico**. Ele serve como um ponto de partida para uma avaliação mais aprofundada.
> 
> **Probabilidade de Acerto do Diagnóstico:**
> 
> Considerando as informações fornecidas e a idade da criança, a probabilidade de acerto deste diagnóstico preliminar é de aproximadamente **60-70%**. É importante confirmar este diagnóstico com profissionais especializados.
> 
> **Próximos Passos:**
> 
> 1.  **Consulta com um especialista:** O ideal é buscar um neuropediatra ou um psiquiatra infantil. Esses profissionais são capacitados para realizar uma avaliação completa e, se necessário, indicar o tratamento adequado.
> 2.  **Avaliação multidisciplinar:** Em alguns casos, pode ser interessante buscar também a avaliação de outros profissionais, como psicólogos, fonoaudiólogos ou terapeutas ocupacionais.
> 3.  **Observação contínua:** Continue observando o comportamento do seu filho(a) em diferentes situações e anote suas percepções para compartilhar com os profissionais.
> 
> **Fontes de Referência:**
> 
> *   **Associação Brasileira de Déficit de Atenção (ABDA):** [https://tdah.org.br/](https://tdah.org.br/)
> *   **DSM-5:** Manual Diagnóstico e Estatístico de Transtornos Mentais (para informações mais técnicas).
> 
> Lembre-se: o diagnóstico precoce e o tratamento adequado podem fazer toda a diferença no desenvolvimento da criança. Não hesite em buscar ajuda profissional!


-------------------------------------------------------------------

 Obrigada por utilizar o sistema de Apoio Inicial ao Diagnóstico de TDAH e TEA para crianças de 3 a 10 anos.

 Fim do programa!
